<a href="https://colab.research.google.com/github/ThumbRocket/AI_Expert_Lecture_Files/blob/main/Lab2_2_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from torch.utils.data import DataLoader

from torchvision.datasets import MNIST
from torchvision.transforms import Compose,ToTensor

from google.colab import drive
import numpy as np
import matplotlib.pyplot as plt
import PIL

#### Connect to local google drive & settings for export/import models

In [ ]:
drive.mount('/content/gdrive')
!mkdir ./gdrive/'My Drive'/MNIST_models/

Mounted at /content/gdrive
mkdir: cannot create directory ‘./gdrive/My Drive/MNIST_models/’: File exists


#### DNN model

In [ ]:
class MNISTDNN(nn.Module):
    def __init__(self,IMG_SIZE=28):
        super(MNISTDNN,self).__init__()
        self.fc1 = nn.Linear(IMG_SIZE*IMG_SIZE,32)
        self.BN1 = torch.nn.BatchNorm1d(32)
        self.fc2 = nn.Linear(32,10)

    def forward(self,x):
        x = self.fc1(x)
        x = F.relu(x)
        x = self.BN1(x)
        x = self.fc2(x)
        x = torch.softmax(x,dim=-1)
        return x

#### CNN model

In [ ]:
class MNISTCNN(nn.Module):
    def __init__(self,IMG_SIZE=28):
        super(MNISTCNN,self).__init__()
        self.conv1 = nn.Conv2d(1,8,5,stride=2)
        self.BN1 = torch.nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(8,8,5,stride=2)
        self.BN2 = torch.nn.BatchNorm2d(8)
        self.conv3 = nn.Conv2d(8,8,3,stride=1)
        self.fc = nn.Linear(8*2*2,10)

    def forward(self,x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.BN1(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.BN2(x)
        x = self.conv3(x)
        x = F.relu(x)
        x = x.view(-1,8*2*2)
        x = self.fc(x)
        x = torch.softmax(x,dim=-1)
        return x

#### Util function for calculating accuracy

In [ ]:
def compute_acc(argmax,y):
    count = 0
    for i in range(len(argmax)):
        if argmax[i]==y[i]:
            count+=1
    return count / len(argmax)

#### hyperparameters & datasets

In [ ]:
IMG_SIZE = 28
BATCH_SIZE = 256
LEARNING_RATE = 0.001
NUM_EPOCHES = 5

In [ ]:
transforms = Compose([
    ToTensor(),
])

trainset = MNIST('/content/gdrive/My Drive/MNIST_models/',train=True,transform=transforms,download=True)
testset = MNIST('/content/gdrive/My Drive/MNIST_models/',train=False,transform=transforms,download=True)

args = {
    'num_workers' : 1,
    'batch_size' : BATCH_SIZE,
    'shuffle' : True,
}

train_loader = DataLoader(trainset,**args)
test_loader = DataLoader(testset,**args)

####Training part(DNN)

In [ ]:
model = MNISTDNN(IMG_SIZE).cuda()

model_parameters = filter(lambda p: p.requires_grad, model.parameters())
num_params = sum([np.prod(p.size()) for p in model_parameters])
print("number of parameters : {}".format(num_params))

optimizer = Adam(model.parameters(),lr=LEARNING_RATE)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(NUM_EPOCHES):
    tot_loss = 0.0

    for x,y in train_loader:
        optimizer.zero_grad()
        x = x.cuda().view(-1,IMG_SIZE*IMG_SIZE)
        y_ = model(x)
        loss = loss_fn(y_, y.cuda())
        loss.backward()
        tot_loss+=loss.item()
        optimizer.step()

    print("Epoch {}, Loss(train) : {}".format(epoch+1,tot_loss/BATCH_SIZE))
    if epoch % 2 == 1:
        x,y = next(iter(test_loader))
        x = x.cuda().view(-1,IMG_SIZE*IMG_SIZE)
        y_ = model(x)
        print(y_)
        _, argmax = torch.max(y_,dim=-1)
        test_acc = compute_acc(argmax,y.numpy())

        print("Acc(val) : {}".format(test_acc))

torch.save(model.state_dict(), "/content/gdrive/My Drive/MNIST_models/DNN.pt")

number of parameters : 25514
Epoch 1, Loss(train) : 1.5983231295831501
Epoch 2, Loss(train) : 1.4412961909547448
tensor([[2.0298e-04, 9.8614e-01, 2.0389e-04,  ..., 3.2311e-03, 4.9533e-04,
         3.9241e-04],
        [3.9084e-07, 1.9736e-05, 9.9878e-01,  ..., 2.1718e-06, 6.9884e-06,
         4.8000e-07],
        [1.2801e-03, 1.5659e-03, 9.5784e-01,  ..., 2.3763e-04, 8.3274e-03,
         2.0782e-06],
        ...,
        [4.0185e-02, 2.5020e-04, 6.9032e-01,  ..., 2.8699e-03, 5.3381e-02,
         6.0479e-04],
        [1.7291e-05, 2.9052e-05, 1.7527e-03,  ..., 7.4845e-06, 8.0926e-05,
         4.8343e-06],
        [5.3739e-05, 1.8445e-05, 9.0929e-04,  ..., 1.3127e-05, 7.4693e-06,
         5.0334e-06]], device='cuda:0', grad_fn=<SoftmaxBackward0>)
Acc(val) : 0.921875
Epoch 3, Loss(train) : 1.4139908510260284
Epoch 4, Loss(train) : 1.400480620097369
tensor([[6.2699e-07, 1.9060e-06, 4.3531e-06,  ..., 9.9993e-01, 1.1307e-06,
         2.5270e-06],
        [9.9970e-01, 3.0968e-07, 9.2141e-05,  

In [ ]:
list(model.parameters())[3].requires_grad

True

In [ ]:
list(model.parameters())[0].size()

torch.Size([32, 784])

In [ ]:
np.prod(list(model.parameters())[0].size())

25088

In [ ]:
# self.fc1(x) 784 * 32
# F.relu(x) 32
# self.BN1(x) 32
# self.fc2(x) 10 * 32
# torch.softmax(x,dim=-1) 10

for i in model.parameters():
  print(i.size())

torch.Size([32, 784])
torch.Size([32])
torch.Size([32])
torch.Size([32])
torch.Size([10, 32])
torch.Size([10])


In [ ]:
model_test = MNISTDNN(IMG_SIZE).cuda()
model_test.load_state_dict(torch.load("/content/gdrive/My Drive/MNIST_models/DNN.pt"))
model_test.eval()
x,y = next(iter(test_loader))
x = x.cuda().view(-1,IMG_SIZE*IMG_SIZE)
y_ = model_test(x)
_, argmax = torch.max(y_,dim=-1)
test_acc = compute_acc(argmax,y.numpy())

print("Acc(test) : {}".format(test_acc))

Acc(test) : 0.94140625


#### Training part(CNN)

In [ ]:
model = MNISTCNN(IMG_SIZE).cuda()

model_parameters = filter(lambda p: p.requires_grad, model.parameters())
num_params = sum([np.prod(p.size()) for p in model_parameters])
print("number of parameters : {}".format(num_params))

optimizer = Adam(model.parameters(),lr=LEARNING_RATE)
loss_fn = nn.CrossEntropyLoss()

for epoch in range(NUM_EPOCHES):
    tot_loss = 0.0

    for x,y in train_loader:
        optimizer.zero_grad()
        x = x.cuda()
        y_ = model(x)
        loss = loss_fn(y_, y.cuda())
        loss.backward()
        tot_loss+=loss.item()
        optimizer.step()

    print("Epoch {}, Loss(train) : {}".format(epoch+1,tot_loss/BATCH_SIZE))
    if epoch % 2 == 1:
        model.eval()

        x,y = next(iter(test_loader))
        x = x.cuda()
        y_ = model(x)
        _, argmax = torch.max(y_,dim=-1)
        test_acc = compute_acc(argmax,y.numpy())

        print("Acc(test) : {}".format(test_acc))

        model.train()

torch.save(model.state_dict(), "/content/gdrive/My Drive/MNIST_models/CNN.pt")

number of parameters : 2762
Epoch 1, Loss(train) : 1.6665414762683213
Epoch 2, Loss(train) : 1.4154077447019517
Acc(test) : 0.95703125
Epoch 3, Loss(train) : 1.391095886938274
Epoch 4, Loss(train) : 1.3814751291647553
Acc(test) : 0.9921875
Epoch 5, Loss(train) : 1.3761286726221442


In [ ]:
model_test = MNISTCNN(IMG_SIZE).cuda()
model_test.load_state_dict(torch.load("/content/gdrive/My Drive/MNIST_models/CNN.pt"))
model_test.eval()
x,y = next(iter(test_loader))
x = x.cuda()
y_ = model_test(x)
_, argmax = torch.max(y_,dim=-1)
test_acc = compute_acc(argmax,y.numpy())

print("Acc(test) : {}".format(test_acc))

Acc(test) : 0.95703125


In [ ]:
for i in model.parameters():
  print(i.size())

torch.Size([8, 1, 5, 5])
torch.Size([8])
torch.Size([8])
torch.Size([8])
torch.Size([8, 8, 5, 5])
torch.Size([8])
torch.Size([8])
torch.Size([8])
torch.Size([8, 8, 3, 3])
torch.Size([8])
torch.Size([10, 32])
torch.Size([10])
